# Notes

TODO

In [12]:
import os 
import re
import csv
import json
import math
from itertools import chain
from datetime import datetime

import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

import tsaug
from tsaug.visualization import plot
import matplotlib.pyplot as plt
plt.style.use('_mpl-gallery')

import tsaug
from tsaug.visualization import plot

# Constants

In [13]:
SEQ_MAX_LEN = 600
NUM_CLASSES = 6
NUM_EXERCISES = 9
NUM_FLAG_BS = 2

In [14]:
dir_root = os.path.join('..')
dir_data_root = os.path.join(dir_root, 'data')
dir_exercises = os.path.join(dir_data_root, 'json', 'patients')
dir_exercises_augmented = os.path.join(dir_data_root, 'json', 'patients_augmented')
dir_exercises_test = os.path.join(dir_data_root, 'json', 'test')

In [15]:
dir_models = os.path.join(dir_root, 'models')
dir_model = os.path.join(dir_models, 'type4-with-fe_no-aug_002')

# Data prep

### Build training set

In [16]:
from utils.input import setup, exercise_to_input

In [17]:
setup(os.path.join('..'), SEQ_MAX_LEN, NUM_EXERCISES, NUM_FLAG_BS)

{'ROOT_PATH': '..',
 'ALL_REGIONS': ['frontal', 'orbital', 'oral'],
 'REGIONS': {'LefteyeMidbottom': 'orbital',
  'LefteyeMidtop': 'orbital',
  'LefteyeInnercorner': 'orbital',
  'LefteyeOutercorner': 'orbital',
  'LefteyebrowInner': 'frontal',
  'LefteyebrowCenter': 'frontal',
  'RighteyeMidbottom': 'orbital',
  'RighteyeMidtop': 'orbital',
  'RighteyeInnercorner': 'orbital',
  'RighteyeOutercorner': 'orbital',
  'RighteyebrowInner': 'frontal',
  'RighteyebrowCenter': 'frontal',
  'NoseTip': 'frontal',
  'MouthLowerlipMidbottom': 'oral',
  'MouthLeftcorner': 'oral',
  'MouthRightcorner': 'oral',
  'MouthUpperlipMidtop': 'oral',
  'ChinCenter': 'oral',
  'ForeheadCenter': 'frontal',
  'LeftcheekCenter': 'oral',
  'RightcheekCenter': 'oral'},
 'BASES': {'LefteyeMidbottom': 'ForeheadCenter',
  'LefteyeMidtop': 'ForeheadCenter',
  'LefteyeInnercorner': 'ForeheadCenter',
  'LefteyeOutercorner': 'ForeheadCenter',
  'LefteyebrowInner': 'ForeheadCenter',
  'LefteyebrowCenter': 'ForeheadCenter

In [18]:
exercises_sources = [
    dir_exercises_test
    #dir_exercises,
    #dir_exercises_augmented
]

In [22]:
feature_engineering_setting = {
    'coordinates': True,
    'normalize_by_start': False,
    'normalize': False,
    'direction': False,
    'distance': False,
    'symetry_diference': False,
    're_base': True,
    'transformation_to_rebase': False,
    'extended_meta': True,
}

In [23]:
count = 0
for exercise_source in exercises_sources:
    for file_name in os.listdir(exercise_source):
        count += 1

test = int(count*0.2)
training_all = count - test
training = int(training_all*0.8)
validation = training_all - training
print(f'Overall number of examples: {count}')
print(f'Overall number of training examples: {training}')
print(f'Overall number of validation examples: {validation}')
print(f'Overall number of testing examples: {test}')

Overall number of examples: 39
Overall number of training examples: 25
Overall number of validation examples: 7
Overall number of testing examples: 7


In [24]:
xslist_meta = list()
xslist_global = list()
xslist_frontal = list()
xslist_oral = list()
xslist_orbital = list()
yslist = list()

for exercise_source in exercises_sources:
    for file_name in os.listdir(exercise_source):
        file_path = os.path.join(exercise_source, file_name)

        if file_name == '.DS_Store': continue
            
        data = json.load(open(file_path, 'r'))

        _xs_meta, _xs_global, _xs_frontal, _xs_oral, _xs_orbital, _ys = exercise_to_input(file_path, feature_engineering_setting)

        yslist.append(_ys)
        xslist_meta.append(_xs_meta)  
        xslist_global.append(_xs_global)
        xslist_frontal.append(_xs_frontal)
        xslist_oral.append(_xs_oral)
        xslist_orbital.append(_xs_orbital)
            
ys = np.array(yslist, dtype='int')
xs_meta = np.array(xslist_meta, dtype='float32')   
xs_global = np.array(xslist_global, dtype='float32') 
xs_frontal = np.array(xslist_frontal, dtype='float32') 
xs_oral = np.array(xslist_oral, dtype='float32') 
xs_orbital = np.array(xslist_orbital, dtype='float32') 

print(ys.shape)
print(xs_meta.shape)
print(xs_global.shape)
print(xs_frontal.shape)
print(xs_oral.shape)
print(xs_orbital.shape)

(39,)
(39, 52)
(39, 126, 600)
(39, 36, 600)
(39, 42, 600)
(39, 48, 600)


# Analysis

## Load Model

In [25]:
model = tf.keras.models.load_model(dir_model)

In [26]:
model.summary()

Model: "model_53"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_42 (InputLayer)          [(None, 126, 600)]   0           []                               
                                                                                                  
 input_43 (InputLayer)          [(None, 36, 600)]    0           []                               
                                                                                                  
 input_44 (InputLayer)          [(None, 42, 600)]    0           []                               
                                                                                                  
 input_45 (InputLayer)          [(None, 48, 600)]    0           []                               
                                                                                           

## Plot TSR

In [30]:
from TSInterpret.InterpretabilityModels.counterfactual.NativeGuideCF import NativeGuideCF

In [35]:
exp_model=NativeGuideCF(model, backend='PYT', mode='feat', method='dtw_bary_center')

TypeError: __init__() missing 1 required positional argument: 'data'

In [ ]:
from TSInterpret.InterpretabilityModels.Saliency.TSR import TSR

In [29]:
input_x = np.array([
    xs_meta[0],
    xs_global[0],
    xs_frontal[0],
    xs_oral[0],
    xs_orbital[0]])

label_y=int(np.argmax(ys[0]))

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5,) + inhomogeneous part.

In [ ]:
int_mod=TSR(model, input_x.shape[-2],input_x.shape[-1], method='IG',mode='time')
exp=int_mod.explain(item,labels=label,TSR =True)

In [11]:
%matplotlib inline  
int_mod.plot(input_x,exp)

NameError: name 'int_mod' is not defined

__END__